### importing modules

In [1]:
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 10)
plt.rcParams['font.size'] = 15

In [2]:
import numpy as np
import gym
from collections import deque
import random
import pickle

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# from tensorflow.python.keras.layers import Dense
# from tensorflow.python.keras import Sequential
# from tensorflow.python.keras.optimizers import Adam

Using TensorFlow backend.


### some constants and variables

In [4]:
# variables

discount = 0.9
epsilon = 0.1
memory_cap = 999999999
# n_step = 4

### classes for different methods

In [ ]:
class Replay_Default():
    
    def __init__(self, buffer_size=50):
        self.experience = []
        
    

In [ ]:
class Q_Learning:
    
    def __init__(self)

In [295]:
class Q_Learning_Default:
    
    def __init__(self, o_space, a_space, buffer, lr, n_step):
        self.n_step = n_step
        self.a_space = a_space
        self.buffer = buffer
        self.lr = lr
        self.experience = deque(maxlen=memory_cap)
        
        self.clf = Sequential()
        self.clf.add(Dense(64, input_shape=(o_space,), activation="relu"))
        self.clf.add(Dense(64, activation="relu"))
        self.clf.add(Dense(self.a_space, activation="linear"))
        self.clf.compile(loss="mse", optimizer=Adam(lr=lr))
        
        self.reset_multistep_exp()
        
    def add_experience(self, s, a, r, s_next, terminal):
        self.multistep_step += 1
        self.multistep_rewards.append(r)
        
        if self.multistep_step == 1:
            self.multistep_state = s
            self.multistep_action = a
        
        if self.multistep_step == self.n_step:
            reward = 0
            
            for i in range(len(self.multistep_rewards)):
                step_reward = self.multistep_rewards[i]
                reward += (discount ** i) * step_reward
                
            self.experience.append([self.multistep_state, self.multistep_action, reward, s_next, terminal])
            self.reset_multistep_exp()
        
    def reset_multistep_exp(self):
        self.multistep_step = 0
        self.multistep_state = None
        self.multistep_action = None
        self.multistep_rewards = []
        
    def get_action(self, s):
        random_explore = np.random.rand()
        
        if random_explore < epsilon:
            return random.randrange(self.a_space)
        
        q_vals = self.clf.predict(s)
        
        return np.argmax(q_vals[0])
    
    def experience_replay(self):
        if len(self.experience) < self.buffer:
            return
        
        exp_batch = random.sample(self.experience, self.buffer)
        
        for s, a, r, s_next, terminal in exp_batch:
            q_update = r
            
            if not terminal:
                q_update = (r + discount * np.amax(self.clf.predict(s_next)[0]))
            
            q_vals = self.clf.predict(s)
            q_vals[0][a] = q_update
            self.clf.fit(s, q_vals, verbose=0)

In [291]:
# class Q_Learning_Default_NP:
    
#     def __init__(self, o_space, a_space, buffer, lr, n_step, state_shape):
#         self.n_step = n_step
#         self.a_space = a_space
#         self.buffer = buffer
#         self.lr = lr
        
#         self.memory_cap = memory_cap
#         self.st_idx = 0
#         self.memory_size = 0
        
#         state_list_shape = [self.memory_cap] + list(state_shape)
        
#         self.s = np.zeros(state_list_shape, dtype=float)
#         self.a = np.zeros(memory_cap, dtype=int)
#         self.r = np.zeros(memory_cap, dtype=float)
#         self.s_next = np.zeros(state_list_shape, dtype=float)
#         self.terminal = np.zeros(memory_cap, dtype=bool)
                
#         self.clf = Sequential()
#         self.clf.add(Dense(64, input_shape=state_shape, activation="relu"))
#         self.clf.add(Dense(64, activation="relu"))
#         self.clf.add(Dense(self.a_space, activation="linear"))
#         self.clf.compile(loss="mse", optimizer=Adam(lr=lr))
        
#     def add_experience(self, s, a, r, s_next, terminal):
#         self.s[self.st_idx] = s
#         self.a[self.st_idx] = a
#         self.r[self.st_idx] = r
#         self.s_next[self.st_idx] = s_next
#         self.terminal[self.st_idx] = terminal
        
#         if self.memory_size < self.memory_cap:
#             self.memory_size += 1
            
#         self.st_idx += 1
        
#         if self.st_idx >= self.memory_cap:
#             self.st_idx = 0 # resetting at 0
        
#     def get_action(self, s):
#         random_explore = np.random.rand()
        
#         if random_explore < epsilon:
#             return random.randrange(self.a_space)
        
#         q_vals = self.clf.predict(s)
        
#         return np.argmax(q_vals[0])
    
#     def experience_replay(self):
#         min_buffer_len = np.min([self.memory_size, self.buffer])
#         idx = np.random.randint(low=0, high=self.memory_size, size=min_buffer_len, dtype=int)
#         s_batch = self.s[idx]
#         a_batch = self.a[idx]
#         r_batch = self.r[idx]
#         s_next_batch = self.s_next[idx]
#         terminal_batch = self.terminal[idx]

#         for i in range(len(s_batch)):
#             s = np.array([s_batch[i]])
#             a = a_batch[i]
#             r = r_batch[i]
#             s_next = np.array([s_next_batch[i]])
#             terminal = terminal_batch[i]
            
#             q_update = r
            
#             if not terminal:
#                 q_update = (r + discount * np.amax(self.clf.predict(s_next)[0]))
            
#             q_vals = self.clf.predict(s)
#             q_vals[0][a] = q_update
# #             print(type(q_vals))
# #             print(type(s))
#             self.clf.fit(s, q_vals, verbose=0)
            
#         time_d = time.time()
        
#         a_perc = (time_b - time_a) / (time_d - time_a)
#         b_perc = (time_c - time_b) / (time_d - time_a)
#         c_perc = (time_d - time_c) / (time_d - time_a)
# #         d_perc = (time_e - time_d) / (time_e - time_a)

# #         print("a", a_perc*100)
# #         print("b", b_perc*100)
# #         print("c", c_perc*100)
# #         print("d", d_perc*100)


In [27]:
class Q_Learning_HER:
    
    def __init__(self, o_space, a_space, buffer, lr, n_step):
        self.n_step = n_step
        self.a_space = a_space
        self.buffer = buffer
        self.lr = lr
        self.experience = []#deque(maxlen=memory_cap)
        self.her_experience = []#deque(maxlen=memory_cap)
        
        self.clf = Sequential()
        self.clf.add(Dense(64, input_shape=(o_space,), activation="relu"))
        self.clf.add(Dense(64, activation="relu"))
        self.clf.add(Dense(self.a_space, activation="linear"))
        self.clf.compile(loss="mse", optimizer=Adam(lr=lr))
        
    def add_experience(self, s, a, r, s_next, terminal):
        self.experience.append([s, a, r, s, terminal])
        
    def add_her_experience(self, s, a, r, s_next, terminal):
        self.her_experience.append([s, a, r, s, terminal])
        
    def get_action(self, s):
        random_explore = np.random.rand()
        
        if random_explore < epsilon:
            return random.randrange(self.a_space)
        
        q_vals = self.clf.predict(s)
        
        return np.argmax(q_vals[0])
    
    def experience_replay(self):
        min_buffer_len = np.min([len(self.experience), self.buffer])
        
        exp_batch = random.sample(self.experience, min_buffer_len)
        
        for s, a, r, s_next, terminal in exp_batch:
            q_update = r
            
            if not terminal:
                q_update = (r + discount * np.amax(self.clf.predict(s_next)[0]))
            
            q_vals = self.clf.predict(s)
            q_vals[0][a] = q_update
            self.clf.fit(s, q_vals, verbose=0)
            
    def modify_her_list(self):
        new_her_experience = copy.deepcopy(self.her_experience)
        new_her_len = len(new_her_experience)
        her_goal = self.her_experience[-1][3]
        
        for i in range(new_her_len):
            new_her_experience[-1-i][0] = her_goal
            new_her_experience[-1-i][2] = -1.0
            new_her_experience[-1-i][3] = her_goal
            new_her_experience[-1-i][4] = False
            
            if (np.sum(np.abs((new_her_experience[-1-i][4] - her_goal))) == 0):
                new_her_experience[-1-i][2] = 0.0
                new_her_experience[-1-i][4] = True
        
        for hx in new_her_experience:
            self.experience.append(hx)
            
    def reset_her(self):
        self.her_experience = [] # deque(maxlen=memory_cap)


In [28]:
class Q_Learning_HERQ:
    
    def __init__(self, o_space, a_space, buffer, lr, n_step):
        self.n_step = n_step
        self.a_space = a_space
        self.buffer = buffer
        self.lr = lr
        self.experience = deque(maxlen=memory_cap)
        self.her_experience = deque(maxlen=memory_cap)
        
        self.clf = Sequential()
        self.clf.add(Dense(64, input_shape=(o_space,), activation="relu"))
        self.clf.add(Dense(64, activation="relu"))
        self.clf.add(Dense(self.a_space, activation="linear"))
        self.clf.compile(loss="mse", optimizer=Adam(lr=lr))
        
    def add_experience(self, s, a, r, s_next, terminal):
        self.experience.append([s, a, r, s, terminal])
        
    def add_her_experience(self, s, a, r, s_next, terminal):
        self.her_experience.append([s, a, r, s, terminal])
        
    def get_action(self, s):
        random_explore = np.random.rand()
        
        if random_explore < epsilon:
            return random.randrange(self.a_space)
        
        q_vals = self.clf.predict(s)
        
        return np.argmax(q_vals[0])
    
    def experience_replay(self):
        min_buffer_len = np.min([len(self.experience), self.buffer])
        
        exp_batch = random.sample(self.experience, min_buffer_len)
        
        for s, a, r, s_next, terminal in exp_batch:
            q_update = r
            
            if not terminal:
                q_update = (r + discount * np.amax(self.clf.predict(s_next)[0]))
            
            q_vals = self.clf.predict(s)
            q_vals[0][a] = q_update
            self.clf.fit(s, q_vals, verbose=0)
            
    def modify_her_list(self):
        new_her_experience = copy.deepcopy(self.her_experience)
        new_her_len = len(new_her_experience)
        her_goal = self.her_experience[-1][3]
        
        for i in range(new_her_len):
            new_her_experience[-1-i][0] = her_goal
            new_her_experience[-1-i][2] = -1.0
            new_her_experience[-1-i][3] = her_goal
            new_her_experience[-1-i][4] = False
            
            if (np.sum(np.abs((new_her_experience[-1-i][4] - her_goal))) == 0):
                new_her_experience[-1-i][2] = 0.0
                new_her_experience[-1-i][4] = True
        
        for hx in new_her_experience:
            self.experience.append(hx)
            
    def reset_her(self):
        self.her_experience = deque(maxlen=memory_cap)


In [ ]:
import time

In [298]:
class Q_Learning_Simple:
    
    def __init__(self, o_space, a_space, buffer, lr, n_step, state_shape):
        self.n_step = n_step
        self.a_space = a_space
        self.buffer = buffer
        self.lr = lr
        
        self.memory_cap = memory_cap
        self.st_idx = 0
        self.memory_size = 0
        
        print(state_shape)
        
        state_list_shape = [self.memory_cap] + list(state_shape)
        
        self.s = np.zeros(state_list_shape, dtype=float)
        self.a = np.zeros(memory_cap, dtype=int)
        self.r = np.zeros(memory_cap, dtype=float)
        self.s_next = np.zeros(state_list_shape, dtype=float)
        self.terminal = np.zeros(memory_cap, dtype=bool)
        
        print(self.s)
        print(self.a)
        print(self.r)
        print(self.s_next)
        print(self.terminal)
                
        self.clf = Sequential()
        self.clf.add(Dense(64, input_shape=state_shape, activation="relu"))
        self.clf.add(Dense(64, activation="relu"))
        self.clf.add(Dense(self.a_space, activation="linear"))
        self.clf.compile(loss="mse", optimizer=Adam(lr=lr))
        
    def add_experience(self, s, a, r, s_next, terminal):
        self.s[self.st_idx] = s
        self.a[self.st_idx] = a
        self.r[self.st_idx] = r
        self.s_next[self.st_idx] = s_next
        self.terminal[self.st_idx] = terminal
        
        if self.memory_size < self.memory_cap:
            self.memory_size += 1
            
        self.st_idx += 1
        
        if self.st_idx >= self.memory_cap:
            self.st_idx = 0 # resetting at 0
        
    def get_action(self, s):
        random_explore = np.random.rand()
        
        if random_explore < epsilon:
            return random.randrange(self.a_space)
        
        q_vals = self.clf.predict(s)
        
        return np.argmax(q_vals[0])
    
    def experience_replay(self):
        
#         print(self.experience)
#         exp_np = np.array(self.experience)
#         print(type(exp_np))
#         print(exp_np)
        
        time_a = time.time()
        min_buffer_len = np.min([self.memory_size, self.buffer])
#         print("min buffer", min_buffer_len)
        time_b = time.time()
#         exp_batch = random.sample(exp_np, min_buffer_len)
#         print(exp_batch)
#         print(0, len(self.experience), min_buffer_len)
        idx = np.random.randint(low=0, high=self.memory_size, size=min_buffer_len, dtype=int)
#         print(idx)
#         print(exp_np)
        s_batch = self.s[idx]
        a_batch = self.a[idx]
        r_batch = self.r[idx]
        s_next_batch = self.s_next[idx]
        terminal_batch = self.terminal[idx]
        
        time_c = time.time()
        
#         print("s_batch 1 shape", s_batch)

        for i in range(len(s_batch)):
            s = np.array([s_batch[i]])
            a = a_batch[i]
            r = r_batch[i]
            s_next = np.array([s_next_batch[i]])
            terminal = terminal_batch[i]
            
#             print(s)
#             print(s_next)
        
            q_update = r
            
#             s_next
            
#             print("s_next", s_next)
#             print("s_next shape", s_next.shape)
            
            if not terminal:
                q_update = (r + discount * np.amax(self.clf.predict(s_next)[0]))
            
            q_vals = self.clf.predict(s)
            q_vals[0][a] = q_update
#             print(type(q_vals))
#             print(type(s))
            self.clf.fit(s, q_vals, verbose=0)
            
        time_d = time.time()
        
        a_perc = (time_b - time_a) / (time_d - time_a)
        b_perc = (time_c - time_b) / (time_d - time_a)
        c_perc = (time_d - time_c) / (time_d - time_a)
#         d_perc = (time_e - time_d) / (time_e - time_a)

#         print("a", a_perc*100)
#         print("b", b_perc*100)
#         print("c", c_perc*100)
#         print("d", d_perc*100)


### main

In [284]:
# learning_rates = [0.5, 0.1, 0.01, 0.001, 0.0001]
# buffers = [50, 100, 250, 500]
# methods = ['sarsa', 'exp_sarsa', 'q']
# seeds = np.arange(20)

# max_episodes = 100

# more doable (recommended by Veronica)

learning_rates = [0.1, 0.01, 0.001]
buffers = [50, 100, 250, 500]
methods = ['sarsa', 'exp_sarsa', 'q']
seeds = np.arange(5)
max_episodes = 100

# learning_rates = [0.1, 0.01, 0.001]
# learning_rates = [0.1, 0.01]
# buffers = [50]
# methods = ['exp_sarsa']
# seeds = np.arange(1)

# max_episodes = 25

n_step = 1

In [285]:
buffer = 50
lr = 0.01
seed = 0
n_step = 1


max_episodes = 100

In [300]:
%%time

max_episodes = 250

print(buffer, lr, seed)
np.random.seed(seed)

eps_steps = []
env = gym.make('CartPole-v1')
o_space = env.observation_space.shape[0]
a_space = env.action_space.n

clf = Q_Learning_Simple(o_space, a_space, buffer, lr, n_step, env.observation_space.shape)

num_episode = 0

for num_episode in range(1, max_episodes+1):# while True:
    start = time.time()
    s = env.reset()
    s = np.reshape(s, [1, o_space])

    num_step = 0
    while True:
        num_step += 1
        
        time_a = time.time()
        a = clf.get_action(s)
        
        time_b = time.time()
        s_next, r, terminal, _ = env.step(a)
        s_next = np.reshape(s_next, [1, o_space])

        if terminal:
            r = -r
        else:
            r = r
        
        time_c = time.time()
        clf.add_experience(s, a, r, s_next, terminal)

        s = s_next

        if terminal:
            print("num_episode: {num_episode}; terminal_step: {num_step}".format(num_episode=num_episode, num_step=num_step))
            eps_steps.append(num_step)
            break
        
        time_d = time.time()
#         clf.experience_replay()
        time_e = time.time()
    
    clf.experience_replay()
        
        
        
#         a_perc = (time_b - time_a) / (time_e - time_a)
#         b_perc = (time_c - time_b) / (time_e - time_a)
#         c_perc = (time_d - time_c) / (time_e - time_a)
#         d_perc = (time_e - time_d) / (time_e - time_a)

#         print("a", a_perc*100)
#         print("b", b_perc*100)
#         print("c", c_perc*100)
#         print("d", d_perc*100)
        
    end = time.time()
    elapsed = end - start
    
#     print("elapsed", elapsed)

50 0.01 0
(4,)
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[0 0 0 ... 0 0 0]
[0. 0. 0. ... 0. 0. 0.]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[False False False ... False False False]
num_episode: 1; terminal_step: 31
num_episode: 2; terminal_step: 8
num_episode: 3; terminal_step: 9
num_episode: 4; terminal_step: 9
num_episode: 5; terminal_step: 13
num_episode: 6; terminal_step: 10
num_episode: 7; terminal_step: 11
num_episode: 8; terminal_step: 10
num_episode: 9; terminal_step: 11
num_episode: 10; terminal_step: 11
num_episode: 11; terminal_step: 99
num_episode: 12; terminal_step: 120
num_episode: 13; terminal_step: 16
num_episode: 14; terminal_step: 84
num_episode: 15; terminal_step: 12
num_episode: 16; terminal_step: 14
num_episode: 17; terminal_step: 9
num_episode: 18; terminal_step: 14
num_episode: 19; terminal_step: 9
num_episode: 20; terminal_step: 35
num_episode: 21; termi

num_episode: 223; terminal_step: 64
num_episode: 224; terminal_step: 179
num_episode: 225; terminal_step: 119
num_episode: 226; terminal_step: 222
num_episode: 227; terminal_step: 119
num_episode: 228; terminal_step: 438
num_episode: 229; terminal_step: 155
num_episode: 230; terminal_step: 136
num_episode: 231; terminal_step: 172
num_episode: 232; terminal_step: 170
num_episode: 233; terminal_step: 13
num_episode: 234; terminal_step: 75
num_episode: 235; terminal_step: 10
num_episode: 236; terminal_step: 11
num_episode: 237; terminal_step: 85
num_episode: 238; terminal_step: 381
num_episode: 239; terminal_step: 176
num_episode: 240; terminal_step: 224
num_episode: 241; terminal_step: 9
num_episode: 242; terminal_step: 126
num_episode: 243; terminal_step: 500
num_episode: 244; terminal_step: 215
num_episode: 245; terminal_step: 85
num_episode: 246; terminal_step: 306
num_episode: 247; terminal_step: 163
num_episode: 248; terminal_step: 214
num_episode: 249; terminal_step: 127
num_episod

In [ ]:
print(buffer, lr, seed)
np.random.seed(seed)

eps_steps = []
env = gym.make('CartPole-v1')
o_space = env.observation_space.shape[0]
a_space = env.action_space.n

clf = Q_Learning_HER(o_space, a_space, buffer, lr, n_step)

num_episode = 0

for num_episode in range(1, max_episodes+1):# while True:
    start = time.time()
    s = env.reset()
    s = np.reshape(s, [1, o_space])
    clf.reset_her()

    num_step = 0
    while True:
        num_step += 1

        a = clf.get_action(s)

        s_next, r, terminal, _ = env.step(a)
        s_next = np.reshape(s_next, [1, o_space])

        if terminal:
            r = -r
        else:
            r = r
        
        clf.add_experience(s, a, r, s_next, terminal)
        clf.add_her_experience(s, a, r, s_next, terminal)

        s = s_next

        if terminal:
            print("num_episode: {num_episode}; terminal_step: {num_step}".format(num_episode=num_episode, num_step=num_step))
            eps_steps.append(num_step)
            break

        clf.experience_replay()
    end = time.time()
    elapsed = end - start
    print(elapsed)

In [ ]:
print(buffer, lr, seed)
np.random.seed(seed)

eps_steps = []
env = gym.make('CartPole-v1')
o_space = env.observation_space.shape[0]
a_space = env.action_space.n

clf = Q_Learning_HERQ(o_space, a_space, buffer, lr, n_step)

num_episode = 0

for num_episode in range(1, max_episodes+1):# while True:
    start = time.time()
    s = env.reset()
    s = np.reshape(s, [1, o_space])
    clf.reset_her()

    num_step = 0
    while True:
        num_step += 1

        a = clf.get_action(s)

        s_next, r, terminal, _ = env.step(a)
        s_next = np.reshape(s_next, [1, o_space])

        if terminal:
            r = -r
        else:
            r = r
        
        clf.add_experience(s, a, r, s_next, terminal)
        clf.add_her_experience(s, a, r, s_next, terminal)

        s = s_next

        if terminal:
            print("num_episode: {num_episode}; terminal_step: {num_step}".format(num_episode=num_episode, num_step=num_step))
            eps_steps.append(num_step)
            break

        clf.experience_replay()
    end = time.time()
    elapsed = end - start
    print(elapsed)

In [26]:
print(buffer, lr, seed)
np.random.seed(seed)

eps_steps = []
env = gym.make('CartPole-v1')
o_space = env.observation_space.shape[0]
a_space = env.action_space.n

clf = Q_Learning_HER(o_space, a_space, buffer, lr, n_step)

num_episode = 0

for num_episode in range(1, max_episodes+1):# while True:
    start = time.time()
    s = env.reset()
    s = np.reshape(s, [1, o_space])
    clf.reset_her()

    num_step = 0
    while True:
        num_step += 1

        a = clf.get_action(s)

        s_next, r, terminal, _ = env.step(a)
        s_next = np.reshape(s_next, [1, o_space])

        if terminal:
            r = -r
        else:
            r = r
        
        clf.add_experience(s, a, r, s_next, terminal)
        clf.add_her_experience(s, a, r, s_next, terminal)

        s = s_next

        if terminal:
            print("num_episode: {num_episode}; terminal_step: {num_step}".format(num_episode=num_episode, num_step=num_step))
            eps_steps.append(num_step)
            break

        clf.experience_replay()
    end = time.time()
    elapsed = end - start
    print(elapsed)

50 0.01 0
num_episode: 1; terminal_step: 10
0.907524824142456
num_episode: 2; terminal_step: 15
0.5968668460845947
num_episode: 3; terminal_step: 14
0.8050639629364014
num_episode: 4; terminal_step: 9
0.6469621658325195
num_episode: 5; terminal_step: 11
0.9324600696563721
num_episode: 6; terminal_step: 17
1.466541051864624
num_episode: 7; terminal_step: 8
0.6855208873748779
num_episode: 8; terminal_step: 34
3.120595932006836
num_episode: 9; terminal_step: 9
0.7865400314331055
num_episode: 10; terminal_step: 17
1.616866111755371


KeyboardInterrupt: 

In [9]:
print(buffer, lr, seed)
np.random.seed(seed)

eps_steps = []
env = gym.make('CartPole-v1')
o_space = env.observation_space.shape[0]
a_space = env.action_space.n

clf = Q_Learning_HER(o_space, a_space, buffer, lr, n_step)

num_episode = 0

for num_episode in range(1, max_episodes+1):# while True:
    s = env.reset()
    s = np.reshape(s, [1, o_space])
    clf.reset_her()

    num_step = 0
    while True:
        num_step += 1

        a = clf.get_action(s)

        s_next, r, terminal, _ = env.step(a)
        s_next = np.reshape(s_next, [1, o_space])

        if terminal:
            r = -r
        else:
            r = r
        
        clf.add_experience(s, a, r, s_next, terminal)
        clf.add_her_experience(s, a, r, s_next, terminal)

        s = s_next

        if terminal:
            print("num_episode: {num_episode}; terminal_step: {num_step}".format(num_episode=num_episode, num_step=num_step))
            eps_steps.append(num_step)
            break

        clf.experience_replay()

50 0.01 0
num_episode: 1; terminal_step: 10
num_episode: 2; terminal_step: 10
num_episode: 3; terminal_step: 15
num_episode: 4; terminal_step: 20
num_episode: 5; terminal_step: 26
num_episode: 6; terminal_step: 62
num_episode: 7; terminal_step: 14
num_episode: 8; terminal_step: 27
num_episode: 9; terminal_step: 26
num_episode: 10; terminal_step: 26
num_episode: 11; terminal_step: 13
num_episode: 12; terminal_step: 13
num_episode: 13; terminal_step: 39
num_episode: 14; terminal_step: 25
num_episode: 15; terminal_step: 16
num_episode: 16; terminal_step: 36
num_episode: 17; terminal_step: 21
num_episode: 18; terminal_step: 11
num_episode: 19; terminal_step: 23
num_episode: 20; terminal_step: 45
num_episode: 21; terminal_step: 48
num_episode: 22; terminal_step: 50
num_episode: 23; terminal_step: 13
num_episode: 24; terminal_step: 28
num_episode: 25; terminal_step: 21
num_episode: 26; terminal_step: 11
num_episode: 27; terminal_step: 9
num_episode: 28; terminal_step: 60
num_episode: 29; ter

In [10]:
eps_steps

[10,
 10,
 15,
 20,
 26,
 62,
 14,
 27,
 26,
 26,
 13,
 13,
 39,
 25,
 16,
 36,
 21,
 11,
 23,
 45,
 48,
 50,
 13,
 28,
 21,
 11,
 9,
 60,
 38,
 16,
 19,
 13,
 12,
 17,
 16,
 17,
 13,
 25,
 33,
 27,
 13,
 19,
 48,
 21,
 23,
 39,
 15,
 20,
 15,
 21,
 38,
 54,
 12,
 42,
 12,
 21,
 22,
 12,
 25,
 18,
 16,
 18,
 21,
 8,
 32,
 22,
 20,
 14,
 10,
 30,
 18,
 27,
 10,
 12,
 38,
 20,
 12,
 22,
 16,
 13,
 15,
 13,
 22,
 51,
 24,
 20,
 20,
 16,
 17,
 13,
 28,
 11,
 15,
 12,
 18,
 30,
 21,
 63,
 18,
 18]

In [297]:
%%time

max_episodes = 250
print(buffer, lr, seed)
np.random.seed(seed)

eps_steps = []
env = gym.make('CartPole-v1')
o_space = env.observation_space.shape[0]
a_space = env.action_space.n

clf = Q_Learning_Default(o_space, a_space, buffer, lr, n_step)

num_episode = 0

for num_episode in range(1, max_episodes+1):# while True:
    s = env.reset()
    s = np.reshape(s, [1, o_space])
#     clf.reset_her()

    num_step = 0
    while True:
        num_step += 1

        a = clf.get_action(s)

        s_next, r, terminal, _ = env.step(a)
        s_next = np.reshape(s_next, [1, o_space])

        if terminal:
            r = -r
        else:
            r = r
        
        clf.add_experience(s, a, r, s_next, terminal)
#         clf.add_her_experience(s, a, r, s_next, terminal)

        s = s_next

        if terminal:
            print("num_episode: {num_episode}; terminal_step: {num_step}".format(num_episode=num_episode, num_step=num_step))
            eps_steps.append(num_step)
            break

    clf.experience_replay()

50 0.01 0
num_episode: 1; terminal_step: 34
num_episode: 2; terminal_step: 102
num_episode: 3; terminal_step: 8
num_episode: 4; terminal_step: 11
num_episode: 5; terminal_step: 10
num_episode: 6; terminal_step: 10
num_episode: 7; terminal_step: 23
num_episode: 8; terminal_step: 10
num_episode: 9; terminal_step: 12
num_episode: 10; terminal_step: 10
num_episode: 11; terminal_step: 124
num_episode: 12; terminal_step: 10
num_episode: 13; terminal_step: 9
num_episode: 14; terminal_step: 10
num_episode: 15; terminal_step: 9
num_episode: 16; terminal_step: 9
num_episode: 17; terminal_step: 9
num_episode: 18; terminal_step: 11
num_episode: 19; terminal_step: 12
num_episode: 20; terminal_step: 18
num_episode: 21; terminal_step: 13
num_episode: 22; terminal_step: 15
num_episode: 23; terminal_step: 16
num_episode: 24; terminal_step: 23
num_episode: 25; terminal_step: 60
num_episode: 26; terminal_step: 30
num_episode: 27; terminal_step: 22
num_episode: 28; terminal_step: 26
num_episode: 29; termi

num_episode: 231; terminal_step: 12
num_episode: 232; terminal_step: 138
num_episode: 233; terminal_step: 140
num_episode: 234; terminal_step: 133
num_episode: 235; terminal_step: 167
num_episode: 236; terminal_step: 177
num_episode: 237; terminal_step: 133
num_episode: 238; terminal_step: 144
num_episode: 239; terminal_step: 131
num_episode: 240; terminal_step: 92
num_episode: 241; terminal_step: 372
num_episode: 242; terminal_step: 159
num_episode: 243; terminal_step: 171
num_episode: 244; terminal_step: 494
num_episode: 245; terminal_step: 175
num_episode: 246; terminal_step: 253
num_episode: 247; terminal_step: 183
num_episode: 248; terminal_step: 202
num_episode: 249; terminal_step: 161
num_episode: 250; terminal_step: 185
CPU times: user 1min 9s, sys: 11.6 s, total: 1min 20s
Wall time: 57.4 s


In [12]:
eps_steps

[55,
 10,
 10,
 14,
 13,
 10,
 25,
 16,
 27,
 24,
 35,
 66,
 12,
 17,
 23,
 18,
 10,
 40,
 32,
 54,
 91,
 50,
 25,
 24,
 47,
 15,
 12,
 16,
 54,
 13,
 23,
 73,
 17,
 20,
 9,
 105,
 106,
 97,
 63,
 67,
 111,
 106,
 15,
 10,
 49,
 49,
 103,
 49,
 43,
 19,
 51,
 93,
 47,
 64,
 25,
 143,
 145,
 198,
 112,
 165,
 12,
 15,
 11,
 12,
 8,
 9,
 10,
 10,
 9,
 8,
 10,
 9,
 9,
 10,
 8,
 10,
 8,
 10,
 11,
 10,
 9,
 28,
 9,
 9,
 12,
 18,
 35,
 39,
 20,
 74,
 15,
 68,
 10,
 10,
 18,
 17,
 30,
 33,
 13,
 10]

In [13]:
print(buffer, lr, seed)
np.random.seed(seed)

eps_steps = []
env = gym.make('MountainCar-v0')
o_space = env.observation_space.shape[0]
a_space = env.action_space.n

clf = Q_Learning_HER(o_space, a_space, buffer, lr, n_step)

num_episode = 0

for num_episode in range(1, max_episodes+1):# while True:
    s = env.reset()
    s = np.reshape(s, [1, o_space])
    clf.reset_her()

    num_step = 0
    while True:
        num_step += 1

        a = clf.get_action(s)

        s_next, r, terminal, _ = env.step(a)
        s_next = np.reshape(s_next, [1, o_space])

        if terminal:
            r = -r
        else:
            r = r
        
        clf.add_experience(s, a, r, s_next, terminal)
        clf.add_her_experience(s, a, r, s_next, terminal)

        s = s_next

        if terminal:
            print("num_episode: {num_episode}; terminal_step: {num_step}".format(num_episode=num_episode, num_step=num_step))
            eps_steps.append(num_step)
            break

        clf.experience_replay()

50 0.01 0
num_episode: 1; terminal_step: 200
num_episode: 2; terminal_step: 200
num_episode: 3; terminal_step: 200
num_episode: 4; terminal_step: 200
num_episode: 5; terminal_step: 200
num_episode: 6; terminal_step: 200
num_episode: 7; terminal_step: 200
num_episode: 8; terminal_step: 200
num_episode: 9; terminal_step: 200
num_episode: 10; terminal_step: 200
num_episode: 11; terminal_step: 200
num_episode: 12; terminal_step: 200
num_episode: 13; terminal_step: 200
num_episode: 14; terminal_step: 200
num_episode: 15; terminal_step: 200
num_episode: 16; terminal_step: 200
num_episode: 17; terminal_step: 200
num_episode: 18; terminal_step: 200
num_episode: 19; terminal_step: 200
num_episode: 20; terminal_step: 200
num_episode: 21; terminal_step: 200
num_episode: 22; terminal_step: 200
num_episode: 23; terminal_step: 200
num_episode: 24; terminal_step: 200
num_episode: 25; terminal_step: 200
num_episode: 26; terminal_step: 200
num_episode: 27; terminal_step: 200
num_episode: 28; terminal_s

In [14]:
eps_steps

[200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200]

In [15]:
print(buffer, lr, seed)
np.random.seed(seed)

eps_steps = []
env = gym.make('MountainCar-v0')
o_space = env.observation_space.shape[0]
a_space = env.action_space.n

clf = Q_Learning_Default(o_space, a_space, buffer, lr, n_step)

num_episode = 0

for num_episode in range(1, max_episodes+1):# while True:
    s = env.reset()
    s = np.reshape(s, [1, o_space])
#     clf.reset_her()

    num_step = 0
    while True:
        num_step += 1

        a = clf.get_action(s)

        s_next, r, terminal, _ = env.step(a)
        s_next = np.reshape(s_next, [1, o_space])

        if terminal:
            r = -r
        else:
            r = r
        
        clf.add_experience(s, a, r, s_next, terminal)
#         clf.add_her_experience(s, a, r, s_next, terminal)

        s = s_next

        if terminal:
            print("num_episode: {num_episode}; terminal_step: {num_step}".format(num_episode=num_episode, num_step=num_step))
            eps_steps.append(num_step)
            break

        clf.experience_replay()

50 0.01 0
num_episode: 1; terminal_step: 200
num_episode: 2; terminal_step: 200
num_episode: 3; terminal_step: 200
num_episode: 4; terminal_step: 200
num_episode: 5; terminal_step: 200
num_episode: 6; terminal_step: 200
num_episode: 7; terminal_step: 200
num_episode: 8; terminal_step: 200
num_episode: 9; terminal_step: 200
num_episode: 10; terminal_step: 200
num_episode: 11; terminal_step: 200
num_episode: 12; terminal_step: 200
num_episode: 13; terminal_step: 200
num_episode: 14; terminal_step: 200
num_episode: 15; terminal_step: 200
num_episode: 16; terminal_step: 200
num_episode: 17; terminal_step: 200
num_episode: 18; terminal_step: 200
num_episode: 19; terminal_step: 200
num_episode: 20; terminal_step: 200
num_episode: 21; terminal_step: 200
num_episode: 22; terminal_step: 200
num_episode: 23; terminal_step: 200
num_episode: 24; terminal_step: 200
num_episode: 25; terminal_step: 200
num_episode: 26; terminal_step: 200
num_episode: 27; terminal_step: 200
num_episode: 28; terminal_s

In [16]:
eps_steps

[200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200]

In [ ]:
##################################